from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
#%tensorflow_version 2.x
#import tensorflow as tf
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

#https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=sXnDmXR7RDr2

# **MyAZNet**  


### **Libraries.** 

In [3]:
import os
import pathlib
import torch
import tarfile
import torchvision
import glob
import torch.nn as nn
from torch.optim import Adam
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

In [4]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from torch.utils.tensorboard import SummaryWriter

In [5]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(device)

cuda


In [7]:
#GPU



### **Parameters.** 

In [8]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
#Dataloader

#Path for training and testing directory
train_path='../MIT_split/train'
test_path='../MIT_split/test'
writer = SummaryWriter('../summaryWriter/')
train_writer = SummaryWriter('../summaryWriter/train')
test_writer = SummaryWriter('../summaryWriter/test')

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=128, shuffle=True
    )

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=128, shuffle=True
    )

In [10]:
#Image categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [11]:
print(classes)

['Opencountry', 'coast', 'forest', 'highway', 'inside_city', 'mountain', 'street', 'tallbuilding']


In [12]:
#CNN Network
class MLP(torch.nn.Module):
    def __init__(self,num_classes=8):
        super(MLP,self).__init__()

        self.num_classes = num_classes

        #Conv1
        self.conv1=torch.nn.Conv2d(3,64,3, padding=1)
        #Shape= (128,3,224,224)
        self.relu1=nn.ReLU()

        #Batch Norm
        self.bn1=torch.nn.BatchNorm2d(64)
        #Shape= (128,12,224,224)
        self.relu2=nn.ReLU()
        #Shape= (128,12,224,224)

        #MaxPool
        self.pool1=torch.nn.MaxPool2d(2)
        #Shape= (128,12,112,112)
        
        #Conv2
        self.conv2=torch.nn.Conv2d(64,64,3)
        #Shape= (32,112,112)
        self.relu3=nn.ReLU()
        #Shape= (32,112,112)

        #Global Avg Pooling
        self.pool2=torch.nn.AdaptiveAvgPool2d(2)

        #Dense
        self.fc=torch.nn.Linear(256*1*1,2048)
        self.relu4=torch.nn.ReLU()

        #Dropout
        self.drop1 = torch.nn.Dropout(p=0.4)

        #Dense
        self.fc2=torch.nn.Linear(2048,8)
        self.softmax = torch.nn.Softmax(dim=1)

       
    #Feed forwad function
        
    def forward(self,input):

        output=self.conv1(input)
        output=self.relu1(output)

        output=self.bn1(output)
        output=self.relu2(output)

        output=self.pool1(output)

        output=self.conv2(output)
        output=self.relu3(output)

        output=self.pool2(output)
        # print(output.shape)

        output=output.view(output.size(0),-1)

        output=self.fc(output)
        output=self.relu4(output)

        output=self.drop1(output)

        output=output.view(output.size(0),-1)

        output=self.fc2(output)            

        output=self.softmax(output)
            
        return output

In [13]:
model=MLP(num_classes=8).to(device)

In [14]:
from torchsummary import summary
summary (model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
       BatchNorm2d-3         [-1, 64, 224, 224]             128
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6         [-1, 64, 110, 110]          36,928
              ReLU-7         [-1, 64, 110, 110]               0
 AdaptiveAvgPool2d-8             [-1, 64, 2, 2]               0
            Linear-9                 [-1, 2048]         526,336
             ReLU-10                 [-1, 2048]               0
          Dropout-11                 [-1, 2048]               0
           Linear-12                    [-1, 8]          16,392
          Softmax-13                    [-1, 8]               0
Total params: 581,576
Trainable params:

In [15]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=100

In [17]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(train_count,test_count)

1881 807


In [ ]:

#Model training and saving best model
#source: https://github.com/gaurav67890/Pytorch_Tutorials/blob/master/cnn-scratch-training.ipynb

#Model training and saving best model
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, data in enumerate(train_loader,0):
        if torch.cuda.is_available():
            train_images, train_labels = data
            train_images=Variable(train_images.cuda())
            train_labels=Variable(train_labels.cuda())
            
        optimizer.zero_grad()
        
        train_outputs=model(train_images)
        loss=loss_function(train_outputs,train_labels)
        loss.backward()
        train_loss += loss.item()*len(train_images)
        optimizer.step()
        
        # train_loss+= loss.cpu().data*train_images.size(0)
        _,prediction=torch.max(train_outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==train_labels.data))

    features = train_images.view(-1, 128 * 128)
    
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count    

    # print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
  
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    val_loss=0.0
    for i, (test_images,test_labels) in enumerate(test_loader):
       if torch.cuda.is_available():
           test_images=Variable(test_images.cuda())
           test_labels=Variable(test_labels.cuda())
            
       test_outputs=model(test_images)
       _,prediction=torch.max(test_outputs.data,1)
       
       temp_test_loss = loss_function(test_outputs, test_labels)
       temp_test_loss.backward()
       val_loss += temp_test_loss.item()*len(test_images)

       test_accuracy+=int(torch.sum(prediction==test_labels.data))
    
    test_accuracy=test_accuracy/test_count
    test_loss=val_loss/test_count
        
    print('Epoch: '+str(epoch+1)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    test_writer.add_scalar('Loss/test', test_loss, epoch+1)
    test_writer.add_scalar('Acc/test', test_accuracy, epoch+1)
    train_writer.add_scalar('Loss/train', train_loss, epoch+1)
    train_writer.add_scalar('Acc/train', train_accuracy, epoch+1)
    
    # Save the best model
    if test_accuracy>best_accuracy:
       torch.save(model.state_dict(),'best_checkpoint.model')
       print("new checkpoint")
       best_accuracy=test_accuracy
writer.add_graph(model, train_images)


In [ ]:
#

# **Hyperparametrization**

In [ ]:
# list all data in history
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../logs/plots_full_data/accuracy_hyper.jpg')
#plt.close()

In [ ]:
  # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../logs/plots_full_data/loss_hyper.jpg')